### FaultFree_Training y FaultFree_Testing

In [2]:
import pyreadr
import pandas as pd
import numpy as np
import os

def load_and_reduce(file_path, n_sims_per_fault=10, n_sims_faultfree=20):
    """
    Carga un archivo .RData, convierte a float32 para ahorrar memoria,
    aplica estratificación por tipo de fallo y devuelve un DataFrame reducido
    manteniendo todas las columnas y sus contenidos.
    """
    try:
        result = pyreadr.read_r(file_path)
        df_name = list(result.keys())[0]
        df = result[df_name]

        # Reducir tipo de dato (float64 -> float32)
        for col in df.select_dtypes(include="float64").columns:
            df[col] = df[col].astype("float32")

        # Añadir etiqueta binaria (sin eliminar nada)
        if 'faultNumber' in df.columns:
            df['fault_present'] = np.where(df['faultNumber'] > 0, 1, 0)

        # Detectar columna de simulación
        sim_col = None
        for candidate in ['simulationRun', 'simRun', 'run']:
            if candidate in df.columns:
                sim_col = candidate
                break
        if sim_col is None:
            raise KeyError("No encontré columna de simulación en el dataset")

        # --- Estratificación ---
        if 'faultNumber' in df.columns:
            # Simulaciones con fallo
            df_faulty = df[df['faultNumber'] > 0]
            selected_faulty_sims = (
                df_faulty.groupby('faultNumber')[sim_col]
                .apply(lambda sims: sims.drop_duplicates().sample(
                    n=min(n_sims_per_fault, len(sims)), random_state=42))
                .reset_index(drop=True)
            )

            # Simulaciones sin fallo
            df_faultfree = df[df['faultNumber'] == 0]
            selected_faultfree_sims = (
                df_faultfree[sim_col]
                .drop_duplicates()
                .sample(n=min(n_sims_faultfree, len(df_faultfree[sim_col].unique())), random_state=42)
            )

            # Combinar IDs de simulaciones seleccionadas
            selected_sims = pd.concat([selected_faulty_sims, selected_faultfree_sims])
            df_reduced = df[df[sim_col].isin(selected_sims)]
        else:
            # Caso FaultFree: solo muestreo de simulaciones
            selected_sims = (
                df[sim_col]
                .drop_duplicates()
                .sample(n=min(n_sims_faultfree, len(df[sim_col].unique())), random_state=42)
            )
            df_reduced = df[df[sim_col].isin(selected_sims)]

        return df_reduced, sim_col

    except Exception as e:
        print(f"Error al cargar {file_path}: {e}")
        return pd.DataFrame(), None


# ------------------------------
# Ejemplo de uso completo
# ------------------------------
DATA_DIR = os.getcwd()
DEST_DIR = os.path.join(DATA_DIR, "DatasetReducido")
os.makedirs(DEST_DIR, exist_ok=True)

file_paths = {
    "Faulty Training": os.path.join(DATA_DIR, "TEP_Faulty_Training.RData"),
    "Faulty Testing": os.path.join(DATA_DIR, "TEP_Faulty_Testing.RData"),
}

for name, path in file_paths.items():
    df_reduced, sim_col = load_and_reduce(path)
    if not df_reduced.empty:
        out_file = os.path.join(DEST_DIR, f"{name.replace(' ', '_')}_reduced.csv")
        df_reduced.to_csv(out_file, index=False)
        print(f"\n--- {name} reducido ---")
        print("Archivo guardado en:", out_file)
        print("Simulaciones:", df_reduced[sim_col].nunique())
        print("Muestras:", len(df_reduced))
        print("Columnas:", df_reduced.columns.tolist())

print("\nTodos los datasets reducidos guardados en la carpeta DatasetReducido.")



--- Faulty Training reducido ---
Archivo guardado en: C:\Users\miren\Documents\GitHub\proyecto_analitica\DatasetReducido\Faulty_Training_reduced.csv
Simulaciones: 10
Muestras: 100000
Columnas: ['faultNumber', 'simulationRun', 'sample', 'xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11', 'fault_present']

--- Faulty Testing reducido ---
Archivo guardado en: C:\Users\miren\Documents\GitHub\proyecto_analitica\DatasetReducido\Faulty_Testing_reduced.csv
Simulaciones: 

### Faulty_Training y Faulty_Testing

In [4]:
import pyreadr
import pandas as pd
import numpy as np
import os

def load_and_reduce(file_path, n_sims_per_fault=10, n_sims_faultfree=20):
    """
    Carga un archivo .RData, convierte a float32 para ahorrar memoria,
    aplica estratificación por tipo de fallo y devuelve un DataFrame reducido
    manteniendo todas las columnas y sus contenidos.
    """
    try:
        result = pyreadr.read_r(file_path)
        df_name = list(result.keys())[0]
        df = result[df_name]

        # Reducir tipo de dato (float64 -> float32)
        for col in df.select_dtypes(include="float64").columns:
            df[col] = df[col].astype("float32")

        # Añadir etiqueta binaria (sin eliminar nada)
        if 'faultNumber' in df.columns:
            df['fault_present'] = np.where(df['faultNumber'] > 0, 1, 0)

        # Detectar columna de simulación
        sim_col = None
        for candidate in ['simulationRun', 'simRun', 'run']:
            if candidate in df.columns:
                sim_col = candidate
                break
        if sim_col is None:
            raise KeyError("No encontré columna de simulación en el dataset")

        # --- Estratificación ---
        if 'faultNumber' in df.columns:
            # Simulaciones con fallo
            df_faulty = df[df['faultNumber'] > 0]
            selected_faulty_sims = (
                df_faulty.groupby('faultNumber')[sim_col]
                .apply(lambda sims: sims.drop_duplicates().sample(
                    n=min(n_sims_per_fault, len(sims)), random_state=42))
                .reset_index(drop=True)
            )

            # Simulaciones sin fallo
            df_faultfree = df[df['faultNumber'] == 0]
            selected_faultfree_sims = (
                df_faultfree[sim_col]
                .drop_duplicates()
                .sample(n=min(n_sims_faultfree, len(df_faultfree[sim_col].unique())), random_state=42)
            )

            # Combinar IDs de simulaciones seleccionadas
            selected_sims = pd.concat([selected_faulty_sims, selected_faultfree_sims])
            df_reduced = df[df[sim_col].isin(selected_sims)]
        else:
            # Caso FaultFree: solo muestreo de simulaciones
            selected_sims = (
                df[sim_col]
                .drop_duplicates()
                .sample(n=min(n_sims_faultfree, len(df[sim_col].unique())), random_state=42)
            )
            df_reduced = df[df[sim_col].isin(selected_sims)]

        return df_reduced, sim_col

    except Exception as e:
        print(f"Error al cargar {file_path}: {e}")
        return pd.DataFrame(), None


# ------------------------------
# Ejemplo de uso completo
# ------------------------------
DATA_DIR = os.getcwd()
DEST_DIR = os.path.join(DATA_DIR, "DatasetReducido")
os.makedirs(DEST_DIR, exist_ok=True)

file_paths = {
    "FaultFree Training": os.path.join(DATA_DIR, "TEP_FaultFree_Training.RData"),
    "FaultFree Testing": os.path.join(DATA_DIR, "TEP_FaultFree_Testing.RData"),
}

for name, path in file_paths.items():
    df_reduced, sim_col = load_and_reduce(path)
    if not df_reduced.empty:
        out_file = os.path.join(DEST_DIR, f"{name.replace(' ', '_')}_reduced.csv")
        df_reduced.to_csv(out_file, index=False)
        print(f"\n--- {name} reducido ---")
        print("Archivo guardado en:", out_file)
        print("Simulaciones:", df_reduced[sim_col].nunique())
        print("Muestras:", len(df_reduced))
        print("Columnas:", df_reduced.columns.tolist())

print("\nTodos los datasets reducidos guardados en la carpeta DatasetReducido.")


--- FaultFree Training reducido ---
Archivo guardado en: C:\Users\miren\Documents\GitHub\proyecto_analitica\DatasetReducido\FaultFree_Training_reduced.csv
Simulaciones: 20
Muestras: 10000
Columnas: ['faultNumber', 'simulationRun', 'sample', 'xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11', 'fault_present']

--- FaultFree Testing reducido ---
Archivo guardado en: C:\Users\miren\Documents\GitHub\proyecto_analitica\DatasetReducido\FaultFree_Testing_reduced.csv
Sim